<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
# Import any packages you need here
# Also define any variables as needed

import numpy as np

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """
    def __init__(self, state, g, h, parent):
        self.state = state # list of lists representing the n x n puzzle
        self.g = g # g(n), path cost from the start node to node n
        self.h = h # h(n), estimated cost of the cheapest path from n to the goal
        self.f = g + h # f(n) = g(n) + h(n), estimated cost of the cheapest solution through n
        self.parent = parent # parent of current node, n
        
    def __lt__(self, other): # 'less than' function for comparision by PriorityQueue
        return self.f < other.f

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
# Add any additional code here (e.g. for the memoization extension)

import numpy as np

def memoize(f):
    '''Code adapted from https://www.python-course.eu/python3_memoization.php'''
    memo = {}
    def wrapper(state, *args, **kwargs):
        if str(state) not in memo:            
            memo[str(state)] = f(state, *args, **kwargs)
        return memo[str(state)]
    return wrapper

# Misplaced tiles heuristic
@memoize
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    n = len(state)
    state = np.array(state)
    goal_state = np.arange(n*n).reshape((n,n))
    h = 0 # number of misplaced tiles
    for i in range(n):
        for j in range(n):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                h += 1  # if the value in the state is different 
                        # from the goal state (ignoring empty tile), 
                        # the tile is misplaced
    return h

# Manhattan distance heuristic
@memoize
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    n = len(state)
    state = np.array(state)
    goal_state = np.arange(n*n).reshape((n,n))
    pos_dict = {} # dictionary to store coordinates of the goal nodes
    for i in range(n):
        for j in range(n):
            pos_dict[goal_state[i][j]] = (i,j)
    h = 0 # Manhattan distance
    for i in range(n):
        for j in range(n): #if it's not the empty tile & states don't match
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]: 
                val = state[i][j] # value of the current node
                h += (abs(pos_dict[val][0] - i) + abs(pos_dict[val][1] - j))
    return h
    
    
# Extra heuristic for the extension.  If implemented, modify the function below
@memoize
def h3(state):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    # Note to professor: I tried various heuristics but none of them was better than the
    # Manhattan distance heuristic 100% of the time so I decided to just leave this blank :(
    return 0

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [3]:
# Import any packages or define any helper functions you need here

from queue import PriorityQueue

def generateChildState(parent_state, direction):
    """This function generates all the possible child states for a given parent state.
    Inputs:
        -parent_state: The parent state as an array of list of lists
        -direction: There are 4 possible directions, 'n', 's', 'e', 'w' (north, south, east, west).
    Outputs:
        -child_state: The resulting child state as an array of list of lists
    """
    i,j = int(np.where(parent_state==0)[0]), int(np.where(parent_state==0)[1])
    child_state = parent_state
    
    if direction == 'n' and i - 1 >= 0: # if the empty tile can go north
        child_state[i,j], child_state[i-1,j] = parent_state[i-1,j], parent_state[i,j]
    elif direction == 's' and i + 1 < len(parent_state): # if the empty tile can go south
        child_state[i,j], child_state[i+1,j] = parent_state[i+1,j], parent_state[i,j] 
    elif direction == 'e' and j + 1 < len(parent_state): # if the empty tile can go east
        child_state[i,j], child_state[i,j+1] = parent_state[i,j+1], parent_state[i,j]
    elif direction == 'w' and j - 1 >= 0: # if the empty tile can go west
        child_state[i,j], child_state[i,j-1] = parent_state[i,j-1], parent_state[i,j]
    
    return child_state

# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    
    '''CHECK FOR ERRORS IN STATE'''
    # if state is not n*n or if there are duplicates in state, return error
    n = len(state)
    state_list = [val for row in state for val in row]
    state_arr = np.array(state)
    if state_arr.shape != (n,n) or len(state_list) != len(set(state_list)):
        return 0,0,0,0,-1
    
    '''
    CHECK IF PUZZLE IS SOLVABLE (adapted from 
    https://www.cs.princeton.edu/courses/archive/fall12/cos226/assignments/8puzzle.html)
    '''
    
    # creating a duplicate of the list that doesn't include 0
    state_list_without_zero = state_list[:] 
    state_list_without_zero.remove(0)
    
    # first scenario - inversions (when an entry is larger than any previous entries)
    inversion_count = 0
    for i in range(len(state_list_without_zero)):
        for j in range(i + 1, len(state_list_without_zero)):
            if state_list_without_zero[j] < state_list_without_zero[i]:
                inversion_count += 1
    # if board size is odd and number of inversions is odd, unsolvable
    if n % 2 == 1 and inversion_count % 2 == 1: 
        return 0,0,0,0,-2
    
    # second scenario (only for puzzles that have an even shape - 2x2, 4x4, 6x6...) 
    # if the row number of the empty tile + inversion count is even, unsolvable
    if n % 2 == 0 and state_list.index(0)//n + inversion_count % 2 == 0:
        return 0,0,0,0,-2
    
    '''INITIALIZING ROOT NODE & OTHER VARIABLES'''
    # root node's params - current state, 0 path cost, heuristic, no parent
    root = PuzzleNode(state_arr, 0, heuristic(state), None)
    n = len(state) # n is the length of the state
    goal = np.arange(n*n).reshape((n,n)) # define goal state
    steps = 0 
    exp = 0
    max_frontier = 1 
    frontier = PriorityQueue()
    frontier.put(root) # initialize frontier with root node
    directions = ['n', 's', 'e', 'w'] # 4 different directions
    states_dict = {str(state_list): 0} # dict that stores visited nodes + their g score
    
    '''ITERATE TILL FRONTIER IS EMPTY'''
    while frontier.empty() == False: # while there are still nodes in the frontier
        parent = frontier.get() # get node with lowest f value
        exp += 1 # number of expanded nodes increases by 1
        if (parent.state == goal).all(): # if puzzle is solved, break loop
            break
        for direction in directions:
            child_state = generateChildState(np.array(parent.state), direction) # generate child state
            if not np.array_equal(child_state, parent.state): # if the empty tile shifted
                # initiate child node (note: g + 1 because path cost increases by 1)
                child = PuzzleNode(child_state, parent.g + 1, heuristic(child_state), parent)
                # convert child state into list in string format
                child_state_list_str = str([val for row in child.state for val in row])
                if child_state_list_str not in states_dict: # if child has not been visited
                    states_dict[child_state_list_str] = child.g # add child to visited nodes dict
                    frontier.put(child) # add child to frontier
                    if frontier.qsize() > max_frontier: # update max frontier length accordingly
                        max_frontier = frontier.qsize()
    
    '''DETERMINE STEP COST & OPTIMAL PATH'''
    final_steps = parent.g # save the final steps before the following steps 
    opt_path = [parent.state.tolist()] # declare optimal path list
    while parent.parent: # while the nodes still have parent nodes, keep iterating till root node
        opt_path.append(parent.parent.state.tolist()) # add parent nodes to optimal path list
        parent = parent.parent
    opt_path.reverse() # reverse list to find proper path
    
    return final_steps, exp, max_frontier, opt_path, 0
    

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [4]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [6]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [7]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [8]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [ ]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [ ]:
## Memoization test - will be carried out after submission